Lets import the required libraries

In [1]:
import numpy as np
import scipy as spy

We need datasets to do this.
A simple data set is a set of images with 64x64 pixel with three color code (RBG)

first we need a library to import images. One such library is pillow.

In [2]:
from PIL import Image

Refer to the doucumentation of Image.
https://pillow.readthedocs.io/en/stable/reference/Image.html

Now that we have our dataset of images and we need to compress it to the required 64x64 size
using our Image module that we have imported.

There is function called "thumbnail" which gives a lower resoulution picture.
Also that we need RBG values of these thumbnails.

In [3]:
import glob,os

In [4]:
size = (64,64)
dataset = []
train_set_y = []
for infile in glob.glob("./imageset/*.jpg"):
    padded_img = np.full((64,64,3),1)
    file,ext = os.path.splitext(infile)
    if "paper" in file:
        train_set_y += [1.0]
    else:
        train_set_y += [0.0]
    with Image.open(infile) as im:
        im.thumbnail(size)
        rgb = im.convert('RGB')
        rgb_vals = np.array(rgb)
        shape = rgb_vals.shape
        padded_img[:shape[0],:shape[1],:] = rgb_vals
        # One can convert and save this padded array into images if one wants.
        # Also one can convert this whole into a funtion with a parameter(string:a path to images).
        dataset += [padded_img]

dataset = np.array(dataset)
train_set_y = np.array(train_set_y).reshape(1,len(train_set_y))
print(train_set_y)

[[0. 0. 0. ... 1. 1. 1.]]


Lets Try to detect images with papers in it

In [5]:
# Lets start by forming all r's , g's and b's in a single row vector

X = np.full((dataset.shape[0],64*64*3),0)
i = 0
for dat_img in dataset:
    X[i][:] = dat_img.reshape(1,64*64*3)
    i += 1

In [6]:
# Define
#     w is (1,64*64*3) shaped array
#     b is a real number
#
#     z = w.T times X plus b

# Initializing w and b
w = np.full((1,64*64*3),2.0)
b = 2.0

# Also don't forget to standaradize the rgb vector dataset

def evaluate(w,b,X):
    shape = X.shape
    # Although we need to divide by standard deviation, 128 works fine
    standard = X
    z = np.dot(w,np.transpose(standard)) + b
    sigmond = 1/(1 + np.exp(-z))
    return sigmond

y_hat = evaluate(w,b,X)
print(y_hat.shape)

(1, 1016)


Now let's write the cost function

In [7]:
def cost(y_hat,y):
    assert(y.shape[1] > 0)
    cost_val = (-1/y.shape[1])*( np.dot(np.log(y_hat),y.transpose()) + np.dot(np.log(1 - y_hat),1 - y.transpose()) )
    return cost_val

cost_val = cost(y_hat,train_set_y)
print(cost_val)

[[nan]]


C:\Users\Jasawanth\AppData\Local\Temp\ipykernel_15420\3452308363.py:3: RuntimeWarning: divide by zero encountered in log
  cost_val = (-1/y.shape[1])*( np.dot(np.log(y_hat),y.transpose()) + np.dot(np.log(1 - y_hat),1 - y.transpose()) )


In [9]:
# We will now try to reduce the above cost by iterating many times and getting the perfect parameters with lowest cost
#
# By the way, variable "alpha" is going to be the learning rate

alpha = 0.5


def propagate(w,b,X,y,learn_cycles,learn_rate):
    w_trained = w
    b_trained = b
    for i in range(learn_cycles):
        y_hat = evaluate(w_trained,b_trained,X)
        cost_val = cost(y_hat,y)
        db = (y_hat - y)
        dw = np.dot(db,X)
        w_trained = w_trained - dw * learn_rate
        b_trained = b_trained - (int(db.sum())/X.shape[0]) * learn_rate
        assert(dw.shape == w.shape)
        print(cost_val)
    return w_trained, b_trained

w_trained, b_trained = propagate(w,b,X,train_set_y,5,1)
print(w_trained)
print(b_trained)

C:\Users\Jasawanth\AppData\Local\Temp\ipykernel_15420\3452308363.py:3: RuntimeWarning: divide by zero encountered in log
  cost_val = (-1/y.shape[1])*( np.dot(np.log(y_hat),y.transpose()) + np.dot(np.log(1 - y_hat),1 - y.transpose()) )
C:\Users\Jasawanth\AppData\Local\Temp\ipykernel_15420\1356929710.py:18: RuntimeWarning: overflow encountered in exp
  sigmond = 1/(1 + np.exp(-z))


[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[nan]]
[[-119903. -117201. -112754. ...    -921.    -921.    -921.]]
1.0915354330708662
